In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from itertools import chain
import torch
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib_inline.backend_inline import set_matplotlib_formats

set_matplotlib_formats('pdf', 'svg')

from deepthermal.FFNN_model import fit_FFNN, FFNN, init_xavier
from deepthermal.validation import create_subdictionary_iterator, k_fold_cv_grid, add_dictionary_iterators
from deepthermal.plotting import plot_result, plot_model_1d

from neural_reparam.reparametrization import (
    get_elastic_metric_loss,
    compute_loss_reparam,
)
from neural_reparam.ResNet import ResNet
from neural_reparam.models import ResCNN, BResCNN, CNN
from neural_reparam.reparam_env import get_epsilon_greedy, get_optimal_path, DiscreteReparamEnv, plot_solution_rl
from neural_reparam.reinforcement_learning import fit_dqn_deterministic

import experiments.curves as c1
from so3.dynamic_distance import find_optimal_diffeomorphism

# make reproducible
seed = torch.manual_seed(0)

In [ ]:
# Load data
N =32

x_train = torch.linspace(0,1 , N, requires_grad=True)
q_train = c1.q(x_train.unsqueeze(1).detach())
r_train = c1.r(x_train.unsqueeze(1).detach())

data = TensorDataset(x_train, q_train, r_train)


In [ ]:
0######
DIR = "../figures/curve_1_rl/"
SET_NAME = "dqn_1"
PATH_FIGURES = os.path.join(DIR, SET_NAME)
if not os.path.exists(PATH_FIGURES):
    os.makedirs(PATH_FIGURES)
########


FOLDS = 1
# loss_func = get_elastic_metric_loss(r=c1.r, constrain_cost=1e3, verbose=False)
# no_penalty_loss_func = get_elastic_metric_loss(r=c1.r, constrain_cost=0, verbose=False)
depth = 4
lr_scheduler = lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, mode='min', factor=0.5, patience=50,
                                                                        verbose=True)
env = DiscreteReparamEnv(depth=depth, data=data)
MODEL_PARAMS = {
    "model": [ResNet],
    "input_dimension": [2],
    "output_dimension": [num_actions],
    "activation": ["relu"],
    "n_hidden_layers": [2],
}

# extend the previous dict with the zip of this
MODEL_PARAMS_EXPERIMENT = {
    "neurons": [32],
}
TRAINING_PARAMS = {
    "env": [env],
    "batch_size": [500],
    "choose_action": [get_epsilon_greedy(epsilon=0.05, num_actions=env.num_actions)],
    "C": [10],
    "memory_size":[2000],
    "DDQN" : [False],
}
# extend the previous dict with the zip of this
TRAINING_PARAMS_EXPERIMENT = {
    "optimizer": ["ADAM"],
    "num_epochs": [300],
    "verbose_interval": [30],
    "learning_rate": [0.1],
    "lr_scheduler" : [lr_scheduler], 
}

In [ ]:
# create iterators
model_params_iter_1 = create_subdictionary_iterator(MODEL_PARAMS)
# model_params_iter = chain.from_iterable((model_params_iter_1, model_params_iter_2))

model_exp_iter = create_subdictionary_iterator(MODEL_PARAMS_EXPERIMENT, product=False)
exp_model_params_iter = add_dictionary_iterators(model_exp_iter, model_params_iter_1)

training_params_iter = create_subdictionary_iterator(TRAINING_PARAMS)
training_exp_iter = create_subdictionary_iterator(TRAINING_PARAMS_EXPERIMENT, product=False)
exp_training_params_iter = add_dictionary_iterators(training_exp_iter, training_params_iter)

Do the actual training

In [ ]:
cv_results = k_fold_cv_grid(
    model_params=exp_model_params_iter,
    fit=fit_dqn_deterministic,
    training_params=exp_training_params_iter,
    data=data,
    folds=FOLDS,
    verbose=True,
    trials=1
)

In [ ]:
# Find DP solution 

with torch.no_grad():
    I1_new = find_optimal_diffeomorphism(q0=q_train, q1=r_train, I0=x_train, I1=x_train, depth=depth)



In [ ]:
model = cv_results["models"][0][0],
plot_kwargs = {
    "env": env, 
    "x_train": x_train,
    "y_train": I1_new,
    "x_axis": "t",
    "y_axis": "$\\varphi(t)$",
    "compare_label": "DP solution"
}
plot_result(
    path_figures=PATH_FIGURES,
    **cv_results,
    plot_function=plot_solution_rl,
    function_kwargs=plot_kwargs,
)